In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder

## Assessing the Dataset

In [ ]:
df = pd.read_csv("obesity_data.csv")

df.head()

In [ ]:
df.describe().T

In [ ]:
df.isna().sum()

In [ ]:
df.info()

## Visualizing the data in histogram

In [ ]:
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(exclude=[np.number]).columns.tolist()
fig, ax = plt.subplots(2, len(numerical_features)//2, figsize=(20, 8))
for i, feature in enumerate(numerical_features):
  sns.histplot(df[feature], ax=ax[i//(len(numerical_features)//2), i%(len(numerical_features)//2)], kde=True)
  ax[i//(len(numerical_features)//2), i%(len(numerical_features)//2)].set_title(feature)
plt.tight_layout()
plt.show()

In [ ]:
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(exclude=[np.number]).columns.tolist()
fig, ax = plt.subplots(2, len(numerical_features)//2, figsize=(20, 8))
for i, feature in enumerate(numerical_features):
  row = i // (len(categorical_features)//2)
  col = i % (len(categorical_features)//2)
  sns.boxplot(df[feature], ax=ax[row, col], orient='h')
  ax[row, col].set_title(feature)
plt.tight_layout()
plt.show()

### NCP: How many main meals do you have daily?

In [ ]:
# Dibuat menjadi bilangan bulat
df['NCP'] = df['NCP'].round(0)

In [ ]:
fig, ax = plt.subplots(3, len(categorical_features)//3, figsize=(20, 20))

for i, feature in enumerate(categorical_features):
    row = i // (len(categorical_features)//3)
    col = i % (len(categorical_features)//3)
    sns.countplot(data=df, x=feature, hue=feature, ax=ax[row, col])
    ax[row, col].set_title(f"Countplot for {feature.replace('_', ' ').title()}")
    ax[row, col].tick_params(axis='x', rotation=90)
    ax[row, col].set_xlabel(feature.replace('_', ' ').title())

plt.tight_layout()
plt.show()


In [ ]:
encoder = LabelEncoder()
df[categorical_features] = df[categorical_features].apply(encoder.fit_transform)
df.head()

In [ ]:
# Check data types after encoding
df.info()

In [ ]:
corr = df.corr()
plt.figure(figsize=(14, 14))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
target_corr = corr['NObeyesdad'].sort_values(ascending=False)

labels = target_corr.index

plt.figure(figsize=(10, 6))
sns.barplot(x=target_corr.values, y=labels, palette='viridis', hue=target_corr.values)
plt.axvline(x=0, color='red', linestyle='--')
for i, v in enumerate(target_corr.values):
    plt.text(v + 0.01, i, f"{v:.3f}", color='red', va='center', fontsize=10)
plt.legend(title="Corr Coefficient", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title("Correlation with Target Variable")
plt.xlabel("Correlation Coefficient")
plt.ylabel("Features")
plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

X = df.drop(columns=['NObeyesdad'])
y = df['NObeyesdad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# (select 10 top features)
selector = SelectKBest(score_func=lambda X, y: mutual_info_classif(X, y, random_state=42), k=10)
selector.fit(X_train, y_train)

selected_features = X.columns[selector.get_support()]
print("Selected Features:", selected_features.tolist())
print("Dropped Features:", X.columns[~selector.get_support()].tolist())


In [ ]:
scores = pd.Series(selector.scores_, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=scores.values, y=scores.index, palette="viridis", hue=scores.values)
for i, v in enumerate(scores.values):
    plt.text(v + 0.01, i, f"{v:.3f}", color='red', va='center', fontsize=10)
plt.title("Feature Importance Scores (mutual_info_classif)")
plt.xlabel("Score")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

In [ ]:
# Scale the data using standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[selected_features])
X_test_scaled = scaler.transform(X_test[selected_features])
X_train_scaled = pd.DataFrame(X_train_scaled, columns=selected_features)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=selected_features)



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV


rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced']
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)
best_rf = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
y_pred = best_rf.predict(X_test_scaled)

print("(Random Forest) Accuracy:", accuracy_score(y_test, y_pred))
print("(Random Forest) Classification Report:\n", classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.title("(Random Forest) Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'newton-cg', 'saga'],
    'multi_class': ['multinomial']
}
grid_search = GridSearchCV(estimator=lr, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)
best_lr = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
y_pred_lr = best_lr.predict(X_test_scaled)



print("(Logistic Regression) Accuracy:", accuracy_score(y_test, y_pred_lr))
print("(Logistic Regression) Classification Report:\n", classification_report(y_test, y_pred_lr))
cm = confusion_matrix(y_test, y_pred_lr)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.title(" (Logistic Regression) Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)
best_xgb = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

y_pred_xgb = best_xgb.predict(X_test_scaled)

# Evaluasi model
print("(XG Boost) Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("(XG Boost) Classification Report:\n", classification_report(y_test, y_pred_xgb))
cm = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.title("(XG Boost) Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
import lightgbm as lgb

lgb = lgb.LGBMClassifier(random_state=42, verbose=-1) #verbose=-1 untuk mematikan log output data trainning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}
grid_search = GridSearchCV(estimator=lgb, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)
best_lgb = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
y_pred_lgb = best_lgb.predict(X_test_scaled)

# Evaluasi model
print("(Light GBM) Accuracy:", accuracy_score(y_test, y_pred_lgb))
print("(Light GBM)) Classification Report:\n", classification_report(y_test, y_pred_lgb))
cm = confusion_matrix(y_test, y_pred_lgb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.title("(Light GBM) Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
from catboost import CatBoostClassifier

cb = CatBoostClassifier(random_state=42, verbose=0)
param_grid = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5, 7],
    'border_count': [32, 64, 128]
}

grid_search = GridSearchCV(estimator=cb, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)
best_cb = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
y_pred_cb = best_cb.predict(X_test_scaled)

# Evaluasi model
print("(Cat Boost) Accuracy:", accuracy_score(y_test, y_pred_cb))
print("(Cat Boost) Classification Report:\n", classification_report(y_test, y_pred_cb))
cm = confusion_matrix(y_test, y_pred_cb)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=encoder.classes_, yticklabels=encoder.classes_)
plt.title("(Cat Boost) Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()